In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)

In [141]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader

def custom_loader(file_path: str):
    if file_path.endswith(".pdf"):
        return PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        return TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

loader = DirectoryLoader("personal", glob="**/*", show_progress=True, loader_cls=custom_loader)
docs = loader.load()



 33%|███▎      | 2/6 [04:16<08:33, 128.27s/it]



 86%|████████▌ | 6/7 [00:00<00:00,  6.51it/s]


In [146]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [150]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [155]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [158]:
vector_store.add_documents(docs)

['02d79a62-2ade-4b53-bae9-e6d87658715f',
 '11e12d41-20f7-4753-b57c-f8fee1075ec7',
 '0a11923c-581a-49a4-9b75-7859d9ec00b8',
 'bef2c41e-baec-428c-aa6e-672e610145fc',
 '4143bec2-ab8b-438f-98eb-9384eff683d0',
 'eafaffa6-5a82-465e-8705-b41ac74c74ef',
 'd3c6df86-0fc9-487d-8e95-53dc5e7711db',
 '50ac5db7-b769-422d-a2e4-9bfaf76ac7d0',
 '1e1423b9-049e-4f7d-a57d-70a02819a2b6',
 'b868497c-42c8-4156-8208-fa386d6f4214',
 'feb7dc5c-e2ef-415f-8f67-8c6f6374f6e5',
 'e231a055-be74-46df-b70a-a3ece848c62d',
 '0414128a-aed4-4e9f-a84c-a8e8c13067bc',
 'e922f8e5-4df5-471b-add1-57cac10f0e96',
 'dbb2329e-ebe1-4e01-98be-563b70cfe29f',
 '6982ebbe-63e1-47fa-9699-ce3bd6781751',
 '423ea964-b455-4d0a-8461-3e9cb2dbaa83',
 '05c32ba2-8b72-4d5f-a672-7bbc6be99748',
 'e1646ed0-d662-4a70-a806-7ec65359f1c1',
 'af0fe243-7b18-4c34-8f64-11138ba5611b',
 '11d73459-d5f6-4ab7-acb6-dc2db9da5ace',
 'e7f6c93d-1bf4-49dc-b60c-c5650d2a60f2',
 '6f73ec67-969a-4ce6-a8a5-d64ed36e95d5',
 '05e515a2-bd0b-4d8e-a699-513e067d54b0',
 '3b0857dc-d254-

In [159]:
stats = index.describe_index_stats()

In [160]:
stats

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 37}},
 'total_vector_count': 37}

In [169]:
vector_store.similarity_search("gre", k=4)

[Document(id='27d5e11e-10cc-47e0-ae7d-9dff0f711bee', metadata={'page': 9.0, 'source': 'personal/paper-crash-detection.pdf'}, page_content='                                                                                                    Nabaraj Subedi, Nirajan Paudel, Manish Chhetri, Sudarshan Acharya, Nabin Lamichhane  \nJournal of IoT in Social, Mobile, Analytics, and Cloud, March 2024, Volume 6, Issue 1 63 \n \nThe facial detection system accurately identifies yawning and eye conditions as in \nFigure 10, and the crash false notification triggering mechanism responds as depicted in Figure \n7. \n Conclusion \nThe results obtained from the drowsiness and crash detection system demonstrate its \neffectiveness in improving road safety. By detecting driver drowsiness and accurately \nidentifying crash events while promptly notifying relevant parties, the system facilit ates swift \nemergency response and aids in preventing potential accidents by monitoring the driver\'s facial \ncondi

In [170]:
doc_retriever = vector_store.as_retriever()

tool for pdf reader

In [191]:
# from pypdf import PdfReader
# from langchain_core.tools import tool

# @tool
# def marks_reader(pdf_path: str):
#     """
#     Read the marks of subjects from the pdf and return subject marks
#     """
#     pdf_reader = PdfReader(pdf_path)
#     text = ""
    
#     for page_num in range(len(pdf_reader.pages)):
#         text += pdf_reader.pages[page_num].extract_text()
    
#     return text

text mark reader tool

In [200]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path: str) -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"

In [201]:
tools = [marks_reader]
llm_with_tools = llm.bind_tools(tools)

In [203]:
llm_with_tools.invoke("can you tell me my first year marks using the markreader tool?")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-10T15:35:52.216445Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1705163208, 'load_duration': 37503375, 'prompt_eval_count': 177, 'prompt_eval_duration': 700000000, 'eval_count': 22, 'eval_duration': 964000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-285bafbe-fd71-452e-b19b-dcbad821fbbf-0', tool_calls=[{'name': 'marks_reader', 'args': {'file_path': 'your_first_year_marks.txt'}, 'id': 'a3e4bf09-9b42-41dc-9c58-42ad34fe1a83', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 22, 'total_tokens': 199})

In [204]:
from langchain.prompts import PromptTemplate

template = """
    You are a helpful assistant that can provide information based on both vector search from a database and other tools.

    ### Task:
    You will be provided with a user query. Your goal is to respond to the query by doing the following:
    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.
    2. If additional context or information is required that can be retrieved from tools (such as reading a PDF), use the appropriate tool.
    3. Combine the information from the database and tools to provide a well-structured and complete response.

    ### User Query:
    {user_query}

    ### Relevant Documents (from Pinecone vector database):
    {retrieved_documents}

    ### Tool Usage:
    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking for marks of certain subjects, use the marks_reader tool to extract information from the document.
    If you used any tools, describe how the tool was used in your response.

    ### Answer:
    Your answer should be concise, clear, and comprehensive, using the retrieved documents and any tool-assisted information.
 """

prompt = PromptTemplate(
    input_variables=["user_query", "retrieved_documents"],
    template=template,
)
print(prompt)

input_variables=['retrieved_documents', 'user_query'] input_types={} partial_variables={} template='\n    You are a helpful assistant that can provide information based on both vector search from a database and other tools.\n\n    ### Task:\n    You will be provided with a user query. Your goal is to respond to the query by doing the following:\n    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.\n    2. If additional context or information is required that can be retrieved from tools (such as reading a PDF), use the appropriate tool.\n    3. Combine the information from the database and tools to provide a well-structured and complete response.\n\n    ### User Query:\n    {user_query}\n\n    ### Relevant Documents (from Pinecone vector database):\n    {retrieved_documents}\n\n    ### Tool Usage:\n    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking fo

In [205]:
chain = prompt | llm_with_tools
print(chain)

first=PromptTemplate(input_variables=['retrieved_documents', 'user_query'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that can provide information based on both vector search from a database and other tools.\n\n    ### Task:\n    You will be provided with a user query. Your goal is to respond to the query by doing the following:\n    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.\n    2. If additional context or information is required that can be retrieved from tools (such as reading a PDF), use the appropriate tool.\n    3. Combine the information from the database and tools to provide a well-structured and complete response.\n\n    ### User Query:\n    {user_query}\n\n    ### Relevant Documents (from Pinecone vector database):\n    {retrieved_documents}\n\n    ### Tool Usage:\n    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, 

In [206]:
user_query = input("Your query :: \n")
retrieved_documents = doc_retriever.invoke(user_query)

response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
print(response.content)

Based on the provided document, I can infer that the author is working on a project related to crash detection for vehicles. The text mentions "Nabaraj Subedi, Nirajan Paudel, Manish Chhetri, Sudarshan Acharya, Nabin Lamichhane" as authors and "Journal of IoT in Social, Mobile, Analytics, and Cloud" as the publication.

The document also discusses the use of Arduino Uno and MPU 6050 components in the project. The author mentions that the Arduino Uno receives velocity and tilt values from the accelerometer and verifies if these values meet predefined conditions before proceeding with further operations.

Unfortunately, I couldn't find any information on marks or grades related to specific subjects in the provided document. If you could provide more context or clarify what you're looking for, I'd be happy to try and assist you further.
